# Pràctica 4 PLH - Rubén Álvarez Aragonés i Pol Pérez Prades

____

# Stuff

## Imports 

In [ ]:
%pip install -r requirements.txt

In [ ]:
import tensorflow as tf
import numpy as np
import nltk
import re
import spacy
from spacy.lang.ca.examples import sentences 
from gensim.models import word2vec
import torch
import plotly.express as px
import pandas as pd
from sklearn.manifold import TSNE

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Requisites
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
import numpy as np


nltk.download('punkt')
nltk.download('stopwords')

spacy.cli.download("ca_core_news_sm")


## Entrenament model Word2Vec

In [ ]:
class Word2VecEmbedder:
    def __init__(self, corpus_path, corpus_size, load_model=False, model_path=None):
        if not load_model:
            self.corpus_path = corpus_path
            self.corpus_size = int(corpus_size * 2**30) if corpus_size else None  # Convert GB to bytes
            self.corpus = self.get_corpus(corpus_path)
            self.fit()
        else:
            try:
                self.load(model_path)
            except FileNotFoundError:
                print("Model not found. Please check the path.")
                return

    def get_corpus(self, corpus_path):
        with open(corpus_path, 'r', encoding='utf-8') as f:
            print("Reading corpus...")
            corpus = f.read(self.corpus_size) if self.corpus_size else f.read()
            print("Preprocessing corpus...")
            corpus = self.preprocess(corpus)  # Preprocess the corpus and tokenize it
        return corpus

    def fit(self, window_size=15, vector_size=300, min_count=10, workers=8, epochs=10):
        # Initialize the Word2Vec model with gensim
        print("Initializing Word2Vec model...")
        self.model = word2vec.Word2Vec(sentences=[self.corpus], vector_size=vector_size, window=window_size, min_count=min_count, workers=workers, epochs=epochs)
        print("Model training completed.")

    def save(self, model_path):
        # Save the model
        self.model.save(model_path)

    def load(self, model_path):
        # Load the model
        self.model = word2vec.Word2Vec.load(model_path)

    def preprocess(self, corpus):
        # Lowercase the corpus
        print("Lowercasing...")
        corpus = corpus.lower()
        
        # Remove special characters
        print("Removing special characters...")
        corpus = re.sub(r'[^a-záàéèíìóòúùñüç\s]', ' ', corpus)
        
        # Tokenize the corpus
        print("Tokenizing...")
        corpus = nltk.word_tokenize(corpus)
        
        # Eliminate last token (probably incomplete word)
        corpus = corpus[:-1]
        
        return corpus

    def get_embedding(self, word):
        # Get the embedding of a word
        try:
            return self.model.wv[word]
        except KeyError:
            print(f"Word '{word}' not in vocabulary.")
            return None

    def print_vocab(self):
        print("Vocabulary:", list(self.model.wv.index_to_key))
        
    

### 1. Model amb 100MB de dades

In [ ]:
Word2Vec_model = Word2VecEmbedder('corpus\catalan_general_crawling.txt', 0.1)

In [ ]:
Word2Vec_model.save('models/word2vec_model.bin')

In [ ]:
Word2Vec_model.print_vocab()

In [ ]:
Word2Vec_model.get_embedding("de")

In [ ]:
Word2Vec_model.model.wv.most_similar("negre")

### 2. Model amb 500MB de dades

In [ ]:
Word2Vec_model_500 = Word2VecEmbedder('corpus\catalan_general_crawling.txt', 0.5)

In [ ]:
Word2Vec_model.get_embedding("hola")

In [ ]:
Word2Vec_model_500.model.wv.most_similar("inshalla")

### 3. Model amb 1GB de dades

In [ ]:
Word2Vec_model_1024 = Word2VecEmbedder('corpus\catalan_general_crawling.txt', 1)

In [ ]:
Word2Vec_model.get_embedding("hola")

### 4. Model amb totes les dades

In [ ]:
Word2Vec_model_full_data = Word2VecEmbedder('corpus\catalan_general_crawling.txt', None)

## Entrenament model de Similitud de Text Semàntic

### Imports

In [5]:
from importació_data import read_all_ts_data, reformat_data, create_corpus, preprocess, flattened_corpus_count, stopwords_cat
from importació_data import pair_list_to_x_y
from model_bàsic import build_and_compile_model_better
import tensorflow as tf
from model_bàsic import compute_pearson

Definim les stopwords del català i la funció del preprocessament del text, que les tindrà en compte a la hora de tokenitzar i natejar el text.

In [6]:
stpw_cat = stopwords_cat()
prepro = lambda x: preprocess(x, stpw_cat)

Per altra banda, llegim les dades i definim les variables més importants per a la creació dels models de similitud de text semàntic.
- Llegim totes les dades de text similarity dividint-les en train, test i val. 
- Reformatejem les dades per a que siguin l'estructura List[Tuple[str, str, float]]. 
- Definim el corpus i el diccionari amb totes les paraules.
- Creem un diccionari de python amb tots els indexs com a claus i amb la repetició de les paraules com a valor.

In [7]:
train, test, val = read_all_ts_data()
train, test, val = reformat_data(train, test, val)
corpus, dictionary = create_corpus(train, test, val, preprocess=prepro)
flat_corpus = flattened_corpus_count(corpus)

### Compartació amb diferents models de Word Embeddings

#### 1. One Hot

In [80]:
from onehot import map_one_hot

Un embedding OneHot té tamany igual a la llargada del diccionari. En la importació de dades ja hem eliminat les stopwords per reduïr la dimensió, però ara també eliminarem del embedding aquelles paraules que es repeteixen masses poques vegades o massa sovint. 

Per aconseguir això creem una llista que conté els indexs de les paraules que sí que utilitzarem i la passem com a argument a la funció *map_one_hot()*, per reduïr la dimensió del embedding.

In [81]:
# Eliminate from the dictionary the words that are repeated very few times or too many times
keys_preprocess = [index  for index in dictionary if flat_corpus[index] > 10 and flat_corpus[index] < 50]

Convertim les paraules a vectors OneHot amb la funció *map_one_hot()*. Aquesta funció crea un vector de zeros de la mida del diccionari i posa un 1 a la posició de la paraula en el diccionari.

In [82]:
mapped_one_hot_train = map_one_hot(train, dictionary, keys_preprocess)
mapped_one_hot_test = map_one_hot(test, dictionary, keys_preprocess)
mapped_one_hot_val = map_one_hot(val, dictionary, keys_preprocess)

Separem el X i Y

In [83]:
x_train_oh, y_train_oh = pair_list_to_x_y(mapped_one_hot_train)
x_val_oh, y_val_oh = pair_list_to_x_y(mapped_one_hot_val)
x_test_oh, y_test_oh = pair_list_to_x_y(mapped_one_hot_test)

Entrenem el model

In [84]:
batch_size_oh: int = 64
num_epochs_oh: int = 64

train_dataset_oh = tf.data.Dataset.from_tensor_slices((x_train_oh, y_train_oh))
train_dataset_oh = train_dataset_oh.shuffle(buffer_size=len(x_train_oh)).batch(batch_size_oh)

val_dataset_oh = tf.data.Dataset.from_tensor_slices((x_val_oh, y_val_oh))
val_dataset_oh = val_dataset_oh.batch(batch_size_oh)

In [85]:
embedding_size_oh = len(keys_preprocess)
model_oh = build_and_compile_model_better(embedding_size = embedding_size_oh)
tf.keras.utils.plot_model(model_oh, show_shapes=True, show_layer_activations=True, )
print(model_oh.summary())

You must install pydot (`pip install pydot`) for `plot_model` to work.


Model: "functional_19"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_18      │ (None, 801)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_19      │ (None, 801)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_27 (Dense)    │ (None, 801)       │    642,402 │ input_layer_18[0… │
│                     │                   │            │ input_layer_19[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_18 (Lambda)  │ (None, 801)       │          0 │ dense_27[0][0],   │
│                     │                   │            │ dense_27[1][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_18          │ (None, 801)       │          0 │ lambda_18[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_28 (Dense)    │ (None, 16)        │     12,832 │ dropout_18[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_19          │ (None, 16)        │          0 │ dense_28[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_29 (Dense)    │ (None, 1)         │         17 │ dropout_19[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_19 (Lambda)  │ (None, 1)         │          0 │ dense_29[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 655,251 (2.50 MB)

 Trainable params: 655,251 (2.50 MB)

 Non-trainable params: 0 (0.00 B)

None


In [86]:
model_oh.fit(train_dataset_oh, epochs=num_epochs_oh, validation_data=val_dataset_oh)

Epoch 1/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.7308 - val_loss: 0.7461
Epoch 2/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.7177 - val_loss: 0.7421
Epoch 3/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.7093 - val_loss: 0.7395
Epoch 4/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.6940 - val_loss: 0.7364
Epoch 5/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.6724 - val_loss: 0.7320
Epoch 6/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.6235 - val_loss: 0.7303
Epoch 7/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.5342 - val_loss: 0.7446
Epoch 8/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.4239 - val_loss: 0.7709
Epoch 9/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.3430 - val_loss: 0.7955
Epoch 10/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.2663 - val_loss: 0.8151
Epoch 11/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.2242 - val_loss: 0.8289
Epoch 12/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.1

Evaluem el model amb la partició de validació

In [87]:
print(f"Correlación de Pearson (train): {compute_pearson(model_oh, x_train_oh, y_train_oh)}")
print(f"Correlación de Pearson (validation): {compute_pearson(model_oh, x_val_oh, y_val_oh)}")

64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
Correlación de Pearson (train): 0.9564202112077503
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Correlación de Pearson (validation): 0.16723982892041112


Provem el model amb la partició de test

In [88]:
print(f"Correlación de Pearson (test): {compute_pearson(model_oh, x_test_oh, y_test_oh)}")

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Correlación de Pearson (test): 0.213597538697998


##### Conclusions de One-hot

Com podem observar a la hora de fer l'entrenament, el model fa molt *overfit*, així que no és capaç de generalitzar bé. Això és degut a que el model OneHot no té en compte la semàntica de les paraules, ja que cada paraula és tractada com a un element únic i no es té en compte el context en el que apareixen les paraules. Una altra raó per la que el model OneHot no és bó és perquè la mida dels vectors és molt gran, ja que la mida dels vectors és igual a la mida del diccionari, i això fa que el model sigui molt ineficient.

#### 2. Word2Vec preentrenats 

In [89]:
from word2vec_tf_idf import map_pairs_w2v

Llegim el model de Word2Vec preentrenat en català.

In [90]:
WORD_EMBEDDING_FILE = "C:/Users/Pol/Downloads/cc.ca.300.bin.gz"


USE_MMAP = False
if USE_MMAP:
    from gensim.models.fasttext import FastTextKeyedVectors
    MMAP_PATH = 'cc.ca.300.bin'
    # wv_model.save(MMAP_PATH)
    wv_model = FastTextKeyedVectors.load(MMAP_PATH, mmap='r')
else:
    from gensim.models import fasttext
    wv_model = fasttext.load_facebook_vectors(WORD_EMBEDDING_FILE)

##### 2.1 Mitjana

Convertim a vectors les frases amb la funció *map_w2v()*. Aquesta funció obté l'embedding de Word2Vec de cada paraula de la frase i en fa la mitjana.

In [91]:
mapped_w2v_mean_train = map_pairs_w2v(train, wv_model, dictionary=dictionary, preprocess=prepro)
mapped_w2v_mean_test = map_pairs_w2v(test, wv_model, dictionary=dictionary, preprocess=prepro)
mapped_w2v_mean_val = map_pairs_w2v(val, wv_model, dictionary=dictionary, preprocess=prepro)

Separem les dades en X i Y.

In [92]:
x_train_w2v_mean, y_train_w2v_mean = pair_list_to_x_y(mapped_w2v_mean_train)
x_val_w2v_mean, y_val_w2v_mean = pair_list_to_x_y(mapped_w2v_mean_val)
x_test_w2v_mean, y_test_w2v_mean = pair_list_to_x_y(mapped_w2v_mean_test)

Entrenem el model

In [93]:
batch_size_w2v_mean: int = 64
num_epochs_w2v_mean: int = 64

train_dataset_w2v_mean = tf.data.Dataset.from_tensor_slices((x_train_w2v_mean, y_train_w2v_mean))
train_dataset_w2v_mean = train_dataset_w2v_mean.shuffle(buffer_size=len(x_train_w2v_mean)).batch(batch_size_w2v_mean)

val_dataset_w2v_mean = tf.data.Dataset.from_tensor_slices((x_val_w2v_mean, y_val_w2v_mean))
val_dataset_w2v_mean = val_dataset_w2v_mean.batch(batch_size_w2v_mean)

In [94]:
embedding_size_w2v_mean = 300
model_w2v_mean = build_and_compile_model_better(embedding_size = embedding_size_w2v_mean)
tf.keras.utils.plot_model(model_w2v_mean, show_shapes=True, show_layer_activations=True)
print(model_w2v_mean.summary())

You must install pydot (`pip install pydot`) for `plot_model` to work.


Model: "functional_21"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_20      │ (None, 300)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_21      │ (None, 300)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_30 (Dense)    │ (None, 300)       │     90,300 │ input_layer_20[0… │
│                     │                   │            │ input_layer_21[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_20 (Lambda)  │ (None, 300)       │          0 │ dense_30[0][0],   │
│                     │                   │            │ dense_30[1][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_20          │ (None, 300)       │          0 │ lambda_20[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_31 (Dense)    │ (None, 16)        │      4,816 │ dropout_20[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_21          │ (None, 16)        │          0 │ dense_31[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_32 (Dense)    │ (None, 1)         │         17 │ dropout_21[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_21 (Lambda)  │ (None, 1)         │          0 │ dense_32[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 95,133 (371.61 KB)

 Trainable params: 95,133 (371.61 KB)

 Non-trainable params: 0 (0.00 B)

None


In [95]:
model_w2v_mean.fit(train_dataset_w2v_mean, epochs=num_epochs_w2v_mean, validation_data=val_dataset_w2v_mean)

Epoch 1/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 12s 72ms/step - loss: 0.7246 - val_loss: 0.7352
Epoch 2/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.7140 - val_loss: 0.7181
Epoch 3/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.6893 - val_loss: 0.6964
Epoch 4/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.6537 - val_loss: 0.6788
Epoch 5/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.6067 - val_loss: 0.6633
Epoch 6/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.5570 - val_loss: 0.6565
Epoch 7/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.4884 - val_loss: 0.6457
Epoch 8/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.4191 - val_loss: 0.6338
Epoch 9/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 0.3542 - val_loss: 0.6304
Epoch 10/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.2989 - val_loss: 0.6156
Epoch 11/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.2593 - val_loss: 0.6262
Epoch 12/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.

Evaluem el model amb la partició de validació

In [96]:
print(f"Correlación de Pearson (train): {compute_pearson(model_w2v_mean, x_train_w2v_mean, y_train_w2v_mean)}")
print(f"Correlación de Pearson (validation): {compute_pearson(model_w2v_mean, x_val_w2v_mean, y_val_w2v_mean)}")

64/64 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step
Correlación de Pearson (train): 0.9872606130377132
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Correlación de Pearson (validation): 0.3928132701809035


Provem el model amb la partició de test

In [97]:
print(f"Correlación de Pearson (test): {compute_pearson(model_w2v_mean, x_test_w2v_mean, y_test_w2v_mean)}")

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Correlación de Pearson (test): 0.33375648147162784


##### Conclusions de Word2Vec preentrenats amb mitjana

Podem observar com amb el model Word2vec i la mitjana el model segueix fent sobreajust però funciona molt millor que el OneHot. Això és degut a que el model Word2Vec té en compte la semàntica de les paraules, ja que les paraules que tenen un significat similar tenen un embedding similar. Això fa que el model sigui capaç de generalitzar millor.

##### 2.2 Tf-idf

Definim el model de Tf-idf amb el corpus definit al inici.

In [98]:
from gensim.models import TfidfModel
modelo_tfidf = TfidfModel(corpus)

Fent servir la funció *map_w2v_tfidf()* s'obté l'embedding de Word2Vec de cada paraula de la frase i s'aplica Tf-idf per obtenir un vector de la frase.

In [101]:
mapped_w2v_tfidf_train = map_pairs_w2v(train, wv_model, dictionary=dictionary,tf_idf_model=modelo_tfidf, preprocess=prepro)
mapped_w2v_tfidf_test = map_pairs_w2v(test, wv_model, dictionary=dictionary,tf_idf_model=modelo_tfidf, preprocess=prepro)
mapped_w2v_tfidf_val = map_pairs_w2v(val, wv_model, dictionary=dictionary,tf_idf_model=modelo_tfidf, preprocess=prepro)

Separem les dades en X i Y.

In [102]:
x_train_w2v_tfidf, y_train_w2v_tfidf = pair_list_to_x_y(mapped_w2v_tfidf_train)
x_val_w2v_tfidf, y_val_w2v_tfidf = pair_list_to_x_y(mapped_w2v_tfidf_val)
x_test_w2v_tfidf, y_test_w2v_tfidf = pair_list_to_x_y(mapped_w2v_tfidf_test)

Entrenem el model

In [103]:
batch_size_w2v_tfidf: int = 64
num_epochs_w2v_tfidf: int = 64

train_dataset_w2v_tfidf = tf.data.Dataset.from_tensor_slices((x_train_w2v_tfidf, y_train_w2v_tfidf))
train_dataset_w2v_tfidf = train_dataset_w2v_tfidf.shuffle(buffer_size=len(x_train_w2v_tfidf)).batch(batch_size_w2v_tfidf)

val_dataset_w2v_tfidf = tf.data.Dataset.from_tensor_slices((x_val_w2v_tfidf, y_val_w2v_tfidf))
val_dataset_w2v_tfidf = val_dataset_w2v_tfidf.batch(batch_size_w2v_tfidf)

In [104]:
embedding_size_w2v_tfidf = 300
model_w2v_tfidf = build_and_compile_model_better(embedding_size = embedding_size_w2v_tfidf)
tf.keras.utils.plot_model(model_w2v_tfidf, show_shapes=True, show_layer_activations=True)
print(model_w2v_tfidf.summary())

You must install pydot (`pip install pydot`) for `plot_model` to work.


Model: "functional_23"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_22      │ (None, 300)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_23      │ (None, 300)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_33 (Dense)    │ (None, 300)       │     90,300 │ input_layer_22[0… │
│                     │                   │            │ input_layer_23[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_22 (Lambda)  │ (None, 300)       │          0 │ dense_33[0][0],   │
│                     │                   │            │ dense_33[1][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_22          │ (None, 300)       │          0 │ lambda_22[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_34 (Dense)    │ (None, 16)        │      4,816 │ dropout_22[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_23          │ (None, 16)        │          0 │ dense_34[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_35 (Dense)    │ (None, 1)         │         17 │ dropout_23[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_23 (Lambda)  │ (None, 1)         │          0 │ dense_35[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 95,133 (371.61 KB)

 Trainable params: 95,133 (371.61 KB)

 Non-trainable params: 0 (0.00 B)

None


In [105]:
model_w2v_tfidf.fit(train_dataset_w2v_tfidf, epochs=num_epochs_w2v_tfidf, validation_data=val_dataset_w2v_tfidf)

Epoch 1/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 40ms/step - loss: 0.7239 - val_loss: 0.7336
Epoch 2/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.7077 - val_loss: 0.7137
Epoch 3/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.6876 - val_loss: 0.6926
Epoch 4/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.6460 - val_loss: 0.6725
Epoch 5/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.5930 - val_loss: 0.6535
Epoch 6/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.5131 - val_loss: 0.6400
Epoch 7/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.4231 - val_loss: 0.6248
Epoch 8/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.3334 - val_loss: 0.6086
Epoch 9/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.2712 - val_loss: 0.6021
Epoch 10/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.2226 - val_loss: 0.5961
Epoch 11/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.1871 - val_loss: 0.6069
Epoch 12/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - loss: 0.

Evaluem el model amb la partició de validació

In [106]:
print(f"Correlación de Pearson (train): {compute_pearson(model_w2v_tfidf, x_train_w2v_tfidf, y_train_w2v_tfidf)}")
print(f"Correlación de Pearson (validation): {compute_pearson(model_w2v_tfidf, x_val_w2v_tfidf, y_val_w2v_tfidf)}")

64/64 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step
Correlación de Pearson (train): 0.9932394105611743
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Correlación de Pearson (validation): 0.45649112450570667


Provem el model amb la partició de test

In [107]:
print(f"Correlación de Pearson (test): {compute_pearson(model_w2v_tfidf, x_test_w2v_tfidf, y_test_w2v_tfidf)}")

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Correlación de Pearson (test): 0.3930336805674309


##### Conclusions de Word2Vec preentrenats amb Tf-idf

Podem observar com, fent us de Tf-idf, el model millora una mica més. Això és degut a que Tf-idf té en compte la freqüència de les paraules en el corpus, i això fa que les paraules que apareixen molt sovint tinguin un pes més baix. Això fa que el model sigui capaç de generalitzar millor. Tot i així el model segueix sobreajustant

#### 3. SpaCy

Importem tot allò necessari per a fer servir el model de SpaCy en català, inclòs el model preentrenat.

In [108]:
import spacy
from spacy_embed import map_spacy_embed

In [109]:
!python -m spacy download ca_core_news_md


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



     ---------------------------------------- 0.0/49.2 MB ? eta -:--:--
     ---------------------------------------- 0.1/49.2 MB 3.3 MB/s eta 0:00:15
     ---------------------------------------- 0.4/49.2 MB 3.3 MB/s eta 0:00:15
      --------------------------------------- 0.9/49.2 MB 5.7 MB/s eta 0:00:09
      --------------------------------------- 1.2/49.2 MB 6.0 MB/s eta 0:00:08
     - -------------------------------------- 1.4/49.2 MB 5.7 MB/s eta 0:00:09
     - -------------------------------------- 1.8/49.2 MB 5.4 MB/s eta 0:00:09
     - -------------------------------------- 2.5/49.2 MB 6.5 MB/s eta 0:00:08
     -- ------------------------------------- 2.8/49.2 MB 6.7 MB/s eta 0:00:07
     -- ------------------------------------- 3.4/49.2 MB 7.1 MB/s eta 0:00:07
     --- ------------------------------------ 4.0/49.2 MB 7.3 MB/s eta 0:00:07
     --- ------------------------------------ 4.8/49.2 MB 7.7 MB/s eta 0:00:06
     ---- ----------------------------------- 5.3/49.2 MB 

Carreguem el model de SpaCy en català.

In [110]:
spacy_model = spacy.load("ca_core_news_md")

Passem les particions de train, test i val a vectors amb la funció *map_spacy()*. Aquesta funció utilitza el model de SpaCy per a convertir les frases a vectors.

In [111]:
mapped_spacy_train = map_spacy_embed(train, spacy_model)
mapped_spacy_test = map_spacy_embed(test, spacy_model)
mapped_spacy_val = map_spacy_embed(val, spacy_model)

Dividim les particions en X i Y.

In [112]:
x_train_sp, y_train_sp = pair_list_to_x_y(mapped_spacy_train)
x_val_sp, y_val_sp = pair_list_to_x_y(mapped_spacy_val)
x_test_sp, y_test_sp = pair_list_to_x_y(mapped_spacy_test)

Entrenem el model

In [113]:
batch_size_sp: int = 64
num_epochs_sp: int = 64

train_dataset_sp = tf.data.Dataset.from_tensor_slices((x_train_sp, y_train_sp))
train_dataset_sp = train_dataset_sp.shuffle(buffer_size=len(x_train_sp)).batch(batch_size_sp)

val_dataset_sp = tf.data.Dataset.from_tensor_slices((x_val_sp, y_val_sp))
val_dataset_sp = val_dataset_sp.batch(batch_size_sp)

In [114]:
embedding_size_sp = 300
model_sp = build_and_compile_model_better(embedding_size = embedding_size_sp)
tf.keras.utils.plot_model(model_sp, show_shapes=True, show_layer_activations=True, )
print(model_sp.summary())

You must install pydot (`pip install pydot`) for `plot_model` to work.


Model: "functional_25"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_24      │ (None, 300)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_25      │ (None, 300)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_36 (Dense)    │ (None, 300)       │     90,300 │ input_layer_24[0… │
│                     │                   │            │ input_layer_25[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_24 (Lambda)  │ (None, 300)       │          0 │ dense_36[0][0],   │
│                     │                   │            │ dense_36[1][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_24          │ (None, 300)       │          0 │ lambda_24[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_37 (Dense)    │ (None, 16)        │      4,816 │ dropout_24[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_25          │ (None, 16)        │          0 │ dense_37[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_38 (Dense)    │ (None, 1)         │         17 │ dropout_25[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_25 (Lambda)  │ (None, 1)         │          0 │ dense_38[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 95,133 (371.61 KB)

 Trainable params: 95,133 (371.61 KB)

 Non-trainable params: 0 (0.00 B)

None


In [115]:
model_sp.fit(train_dataset_sp, epochs=num_epochs_sp, validation_data=val_dataset_sp)

Epoch 1/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 0.7256 - val_loss: 0.7358
Epoch 2/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.7120 - val_loss: 0.7282
Epoch 3/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.6974 - val_loss: 0.7170
Epoch 4/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.6781 - val_loss: 0.7059
Epoch 5/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.6569 - val_loss: 0.6943
Epoch 6/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.6208 - val_loss: 0.6850
Epoch 7/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.6022 - val_loss: 0.6777
Epoch 8/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.5681 - val_loss: 0.6768
Epoch 9/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.5407 - val_loss: 0.6573
Epoch 10/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.5137 - val_loss: 0.6573
Epoch 11/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.4758 - val_loss: 0.6420
Epoch 12/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.4

Evaluem el model amb al partició de validation

In [116]:
print(f"Correlación de Pearson (train): {compute_pearson(model_sp, x_train_sp, y_train_sp)}")
print(f"Correlación de Pearson (validation): {compute_pearson(model_sp, x_val_sp, y_val_sp)}")

64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
Correlación de Pearson (train): 0.9482275546865951
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Correlación de Pearson (validation): 0.324149932695841


Provem el model amb la partició de test

In [117]:
print(f"Correlación de Pearson (train): {compute_pearson(model_sp, x_test_sp, y_test_sp)}")

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Correlación de Pearson (train): 0.2461349343045748


##### Conclusions de SpaCy

Utilitzant el model de SpaCy, el model segueix fent sobreajust, i funciona de manera molt similar al Word2Vec, però una mica pitjor. Això és degut a que el model de SpaCy també té en compte la semàntica de les paraules, ja que les paraules que tenen un significat similar tenen un embedding similar. Això fa que el model sigui capaç de generalitzar millor.

#### 4. RoBERTa 

In [10]:
from roberta_base import map_roberta_embed
import spacy

In [11]:
!python -m spacy download ca_core_news_trf


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/457.1 MB ? eta -:--:--
     ---------------------------------------- 0.2/457.1 MB 3.9 MB/s eta 0:01:57
     ---------------------------------------- 0.5/457.1 MB 5.2 MB/s eta 0:01:28
     ---------------------------------------- 0.9/457.1 MB 6.5 MB/s eta 0:01:10
     ---------------------------------------- 1.4/457.1 MB 7.2 MB/s eta 0:01:03
     ---------------------------------------- 1.8/457.1 MB 7.6 MB/s eta 0:01:01
     ---------------------------------------- 2.4/457.1 MB 8.3 MB/s eta 0:00:55
     ---------------------------------------- 2.7/457.1 MB 8.2 MB/s eta 0:00:56
     ---------------------------------------- 3.2/457.1 MB 8.5 MB/s eta 0:00:54
     ---------------------------------------- 3.7/457.1 MB 8.9 MB/s eta 0:00:52
     ---------------------------------------- 4.1/457.1 MB 9.1 MB/s eta 0:00:50
     ---------------------------------------- 4.7/457.1 MB 9.1 MB/s eta 0:00:50
     ---------------------------------------- 5

Carreguem el model de RoBERTa preentrenat en català.

In [12]:
roberta_model = spacy.load("ca_core_news_trf")

Passem les particions de train, test i val a vectors amb la funció *map_roberta()*. Aquesta funció utilitza el model de RoBERTa per a convertir les frases a vectors.

In [13]:
mapped_roberta_train = map_roberta_embed(train, roberta_model)
mapped_roberta_test = map_roberta_embed(test, roberta_model)
mapped_roberta_val = map_roberta_embed(val, roberta_model)

Dividim les particions en X i Y.

In [14]:
x_train_roberta, y_train_roberta = pair_list_to_x_y(mapped_roberta_train)
x_val_roberta, y_val_roberta = pair_list_to_x_y(mapped_roberta_val)
x_test_roberta, y_test_roberta = pair_list_to_x_y(mapped_roberta_test)

Entrenem el model.

In [15]:
batch_size_roberta: int = 64
num_epochs_roberta: int = 64

train_dataset_roberta = tf.data.Dataset.from_tensor_slices((x_train_roberta, y_train_roberta))
train_dataset_roberta = train_dataset_roberta.shuffle(buffer_size=len(x_train_roberta)).batch(batch_size_roberta)

val_dataset_roberta = tf.data.Dataset.from_tensor_slices((x_val_roberta, y_val_roberta))
val_dataset_roberta = val_dataset_roberta.batch(batch_size_roberta)

In [16]:
embedding_size_roberta = 768
model_roberta = build_and_compile_model_better(embedding_size = embedding_size_roberta)
tf.keras.utils.plot_model(model_roberta, show_shapes=True, show_layer_activations=True)
print(model_roberta.summary())


You must install pydot (`pip install pydot`) for `plot_model` to work.


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 768)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 768)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 768)       │    590,592 │ input_layer[0][0… │
│                     │                   │            │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 768)       │          0 │ dense[0][0],      │
│                     │                   │            │ dense[1][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 768)       │          0 │ lambda[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 16)        │     12,304 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 16)        │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 1)         │         17 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_1 (Lambda)   │ (None, 1)         │          0 │ dense_2[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 602,913 (2.30 MB)

 Trainable params: 602,913 (2.30 MB)

 Non-trainable params: 0 (0.00 B)

None


In [17]:
model_roberta.fit(train_dataset_roberta, epochs=num_epochs_roberta, validation_data=val_dataset_roberta)

Epoch 1/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 0.7264 - val_loss: 0.7412
Epoch 2/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.7196 - val_loss: 0.7335
Epoch 3/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.7089 - val_loss: 0.7231
Epoch 4/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.6908 - val_loss: 0.7103
Epoch 5/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.6668 - val_loss: 0.7003
Epoch 6/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.6353 - val_loss: 0.6926
Epoch 7/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.6061 - val_loss: 0.6760
Epoch 8/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.5609 - val_loss: 0.6610
Epoch 9/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.5258 - val_loss: 0.6500
Epoch 10/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.5042 - val_loss: 0.6386
Epoch 11/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.4783 - val_loss: 0.6312
Epoch 12/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.4

Evaluar el model amb la partició de validació.

In [18]:
print(f"Correlación de Pearson (train): {compute_pearson(model_roberta, x_train_roberta, y_train_roberta)}")
print(f"Correlación de Pearson (validation): {compute_pearson(model_roberta, x_val_roberta, y_val_roberta)}")

64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Correlación de Pearson (train): 0.9432079034985339
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Correlación de Pearson (validation): 0.4552241655201091


Provar el model amb la partició de test.

In [19]:
print(f"Correlación de Pearson (test): {compute_pearson(model_roberta, x_test_roberta, y_test_roberta)}")

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Correlación de Pearson (test): 0.3001994328819157


##### Conclusions Roberta

Veiem com el model de RoBERTa és un dels que millor funciona, ja que és el model més avançat i el que millor semàntica té. Això fa que el model sigui capaç de generalitzar millor.
Per altra banda, els seus vectors són més grans així que és més pesat d'executar que els altres models.

#### 5. RoBERTa fine-tuned

Instal·lem les llibreries necessàries

In [20]:
%pip install tf-keras
%pip install transformers

     ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
     -- ------------------------------------- 0.1/1.7 MB 3.0 MB/s eta 0:00:01
     -------- ------------------------------- 0.3/1.7 MB 4.3 MB/s eta 0:00:01
     -------------- ------------------------- 0.6/1.7 MB 4.8 MB/s eta 0:00:01
     ------------------- -------------------- 0.8/1.7 MB 4.9 MB/s eta 0:00:01
     ------------------------- -------------- 1.1/1.7 MB 4.9 MB/s eta 0:00:01
     ---------------------------- ----------- 1.2/1.7 MB 4.7 MB/s eta 0:00:01
     ------------------------------------ --- 1.6/1.7 MB 5.0 MB/s eta 0:00:01
     ---------------------------------------  1.7/1.7 MB 5.0 MB/s eta 0:00:01
     ---------------------------------------- 1.7/1.7 MB 4.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Fem els imports de les funcions.

In [26]:
from roberta_fine_tuned import prepare_roberta_ft, compute_pearson_roberta_ft, x_y_split_roberta_ft
from transformers import pipeline, AutoTokenizer

ImportError: cannot import name 'compute_pearson_roberta_ft' from 'roberta_fine_tuned' (c:\Users\Pol\Desktop\1_Uni\Q4\PLH\PLH-WordEmbeddings\roberta_fine_tuned.py)

Creem el model de RoBERTa fine-tuned.

In [3]:
model_roberta_ft = 'projecte-aina/roberta-base-ca-v2-cased-sts'
tokenizer_roberta_ft = AutoTokenizer.from_pretrained(model_roberta_ft)
pipe_roberta_ft = pipeline('text-classification', model=model_roberta_ft, tokenizer=tokenizer_roberta_ft)

c:\Users\Pol\Desktop\1_Uni\Q4\PLH\PLH-WordEmbeddings\PLH-WE-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Pol\Desktop\1_Uni\Q4\PLH\PLH-WordEmbeddings\PLH-WE-env\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Pol\.cache\huggingface\hub\models--projecte-aina--roberta-base-ca-v2-cased-sts. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Dev

Separem en X i Y.

In [16]:
X_train_roberta_ft, y_train_roberta_ft = x_y_split_roberta_ft(train)
X_val_roberta_ft, y_val_roberta_ft = x_y_split_roberta_ft(val)
X_test_roberta_ft, y_test_roberta_ft = x_y_split_roberta_ft(test)

In [28]:
X = X_train_roberta_ft + X_val_roberta_ft + X_test_roberta_ft
y = y_train_roberta_ft + y_val_roberta_ft + y_test_roberta_ft

Obtenim els resultats del model

In [29]:
predictions = pipe_roberta_ft(prepare_roberta_ft(X, tokenizer_roberta_ft), add_special_tokens=False)

In [31]:
print(f"Correlación de Pearson (all data): {compute_pearson_roberta_ft(predictions, y)}")

Correlación de Pearson (all data): 0.8905615573801516


In [33]:
print(f"Correlación de Pearson (test): {compute_pearson_roberta_ft(predictions[-len(y_test_roberta_ft):], y_test_roberta_ft)}")

Correlación de Pearson (test): 0.7527686781420264


##### Conclusions Roberta fine-tuned